#### Script para integração dos limites de imóveis rurais do CAR com as bases: Usos ABC, Declividade, Edafo, TI, UCPI, Amazônia Legal e Municípios

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [2]:
# Módulos necessários
import os
import glob
import geopandas as gpd

In [3]:
# Definir diretório principal de Integrações
dirpath = input('Diretório principal: ')

In [4]:
# Definir pasta com os arquivos do CAR
car_path = input('Diretório do CAR: ')

In [ ]:
# Definir caminho da pasta: Usos_ABC_Municipios
uso_path = input('Diretório dos Usos ABC por município: ')

In [ ]:
# Definir caminho da pasta: Decliv_UF_Municipios
decl_path = input('Diretório da Declividade por município: ')

In [ ]:
# Definir caminho da pasta: Aptd_Edafo_Municipios
edafo_path = input('Diretório da Aptidão do Solo por município: ')

In [ ]:
# Definir caminho da pasta: Terras_Indigenas_Municipios
ti_path = input('Diretório da Terra Indígena por município: ')

In [ ]:
# Definir caminho da pasta: UCPI_Municipios
ucpi_path = input('Diretório de UCPI por município: ')

In [ ]:
# Definir caminho da pasta: Amz_Legal_Municipios
amz_path = input('Diretório de Amazônia Legal por município: ')

In [ ]:
# Definir caminho do shapefile de Municípios
mun_path = input('Shapefile de Municípios: ')

In [5]:
# Listar anos de integração
anos = ['2008', '2021']

In [7]:
# Criar pasta para armazenar arquivos temporários
temp_path = os.path.join(dirpath, 'Temporarios')
os.makedirs(temp_path, exist_ok=True)
# Renomeando AREA_IMOVEL para seus respectivos municípios
municipios = []
# Percorre a pasta 'CAR' e suas subpastas
for root, dirs, files in os.walk(car_path):
    for dir in dirs:
        # Verifica se o nome da subpasta é um código de município
        if dir.startswith('SHAPE_'):
            municipio_codigo = dir.split('_')[1]
            municipios.append(municipio_codigo)
            # Constrói o caminho completo para a pasta 'AREA_IMOVEL'
            area_imovel_dir = os.path.join(root, dir, 'AREA_IMOVEL')
            
            # Verifica se a pasta 'AREA_IMOVEL' existe
            if os.path.exists(area_imovel_dir):
                # Renomeia os arquivos dentro de 'AREA_IMOVEL'
                for filename in os.listdir(area_imovel_dir):
                    if filename.startswith('AREA_IMOVEL'):
                        novo_nome = f'{municipio_codigo}_{filename}'
                        arquivo_antigo = os.path.join(area_imovel_dir, filename)
                        novo_arquivo = os.path.join(area_imovel_dir, novo_nome)
                        os.rename(arquivo_antigo, novo_arquivo)
                        print(f'Renomeado: {arquivo_antigo} -> {novo_arquivo}')

In [ ]:
# Interseção 1 - Usos ABC + CAR
for ano in anos:
    # Caminho dos vetores dos Usos ABC
    usos = glob.glob(uso_path + fr'**/*/*_{ano}_*.shp')
    # Caminho dos vetores de limites dos imóveis
    car = glob.glob(car_path + '**/*/*/*/*.shp')
    caminho_usos = []
    caminho_car = []
    for arquivo in usos:
        caminho = os.path.join(uso_path,arquivo)
        caminho_usos.append(caminho)
    for arquivo in car:
        caminho = os.path.join(car_path,arquivo)
        caminho_car.append(caminho)
    vetores = caminho_usos + caminho_car
    for municipio in municipios:
        for vetor in vetores:
            if municipio in vetor and 'Usos' in vetor:
                nome_arq = os.path.basename(vetor).replace('.shp', '_CAR.shp')
                uso = gpd.read_file(vetor)
            if municipio in vetor and 'AREA_IMOVEL' in vetor:
                area = gpd.read_file(vetor)
        base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [ ]:
# Interseção 2 - Interseção 1 + Declividade
for ano in anos:
    # Caminho dos vetores de Interseção 1
    int = glob.glob(temp_path + fr'**/*{ano}*CAR.shp')
    # Caminho dos vetores de Declividade
    base = glob.glob(decl_path + '**/*.shp')
    caminho_int = []
    caminho_base = []
    for arquivo in int:
        caminho = os.path.join(temp_path,arquivo)
        caminho_int.append(caminho)
    for arquivo in base:
        caminho = os.path.join(decl_path,arquivo)
        caminho_base.append(caminho)
    vetores = caminho_int + caminho_base
    for municipio in municipios:
        for vetor in vetores:
            if municipio in vetor and 'Usos' in vetor:
                nome_arq = os.path.basename(vetor).replace('CAR.shp', 'Decl_CAR.shp')
                uso = gpd.read_file(vetor)
            if municipio in vetor and 'Decliv' in vetor:
                area = gpd.read_file(vetor)
        base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))
# Dissolve
base = glob.glob(temp_path + '**/*Decl_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    nome_arq = os.path.basename(arquivo)
    diss = arq.dissolve(by=['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_DECLIV'], as_index=True)
    diss.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [ ]:
# Interseção 3 - Interseção 2 + Edafo
for ano in anos:
    # Caminho dos vetores de Interseção 2
    int = glob.glob(temp_path + fr'**/*{ano}*Decl_CAR.shp')
    # Caminho dos vetores de Edafo
    base = glob.glob(edafo_path + '**/*.shp')
    caminho_int = []
    caminho_base = []
    for arquivo in int:
        caminho = os.path.join(temp_path,arquivo)
        caminho_int.append(caminho)
    for arquivo in base:
        caminho = os.path.join(edafo_path,arquivo)
        caminho_base.append(caminho)
    vetores = caminho_int + caminho_base
    for municipio in municipios:
        for vetor in vetores:
            if municipio in vetor and 'Usos' in vetor:
                nome_arq = os.path.basename(vetor).replace('CAR.shp', 'Aptd_CAR.shp')
                uso = gpd.read_file(vetor)
            if municipio in vetor and 'Aptd' in vetor:
                area = gpd.read_file(vetor)
        base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))
# Dissolve
base = glob.glob(temp_path + '**/*Aptd_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    nome_arq = os.path.basename(arquivo)
    diss = arq.dissolve(by=['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_DECLIV', 'APTD_EDAFO', 'CD_EDAFO', 'LEG_SOLO'], as_index=True)
    diss.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [ ]:
# Interseção 4 - Interseção 3 + TI
for ano in anos:
    # Caminho dos vetores de Interseção 3
    int = glob.glob(temp_path + fr'**/*{ano}*Aptd_CAR.shp')
    # Caminho dos vetores de TI
    base = glob.glob(ti_path + '**/*.shp')
    caminho_int = []
    caminho_base = []
    for arquivo in int:
        caminho = os.path.join(temp_path,arquivo)
        caminho_int.append(caminho)
    for arquivo in base:
        caminho = os.path.join(ti_path,arquivo)
        caminho_base.append(caminho)
    vetores = caminho_int + caminho_base
    for municipio in municipios:
        for vetor in vetores:
            if municipio in vetor and 'Usos' in vetor:
                nome_arq = os.path.basename(vetor).replace('CAR.shp', 'TI_CAR.shp')
                uso = gpd.read_file(vetor)
            if municipio in vetor and 'TI' in vetor:
                area = gpd.read_file(vetor)
        base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))
# Dissolve
base = glob.glob(temp_path + '**/*TI_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    nome_arq = os.path.basename(arquivo)
    diss = arq.dissolve(by=['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_DECLIV', 'APTD_EDAFO', 'CD_EDAFO', 'LEG_SOLO', 'CD_TERRIND'], as_index=True)
    diss.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [ ]:
# Interseção 5 - Interseção 4 + UCPI
for ano in anos:
    # Caminho dos vetores de Interseção 4
    int = glob.glob(temp_path + fr'**/*{ano}*TI_CAR.shp')
    # Caminho dos vetores de UCPI
    base = glob.glob(ucpi_path + '**/*.shp')
    caminho_int = []
    caminho_base = []
    for arquivo in int:
        caminho = os.path.join(temp_path,arquivo)
        caminho_int.append(caminho)
    for arquivo in base:
        caminho = os.path.join(ti_path,arquivo)
        caminho_base.append(caminho)
    vetores = caminho_int + caminho_base
    for municipio in municipios:
        for vetor in vetores:
            if municipio in vetor and 'Usos' in vetor:
                nome_arq = os.path.basename(vetor).replace('CAR.shp', 'UCPI_CAR.shp')
                uso = gpd.read_file(vetor)
            if municipio in vetor and 'UCPI' in vetor:
                area = gpd.read_file(vetor)
        base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))
# Dissolve
base = glob.glob(temp_path + '**/*UCPI_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    nome_arq = os.path.basename(arquivo)
    diss = arq.dissolve(by=['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_DECLIV', 'APTD_EDAFO', 'CD_EDAFO', 'LEG_SOLO', 'CD_TERRIND', 'CD_UCPI'], as_index=True)
    diss.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [ ]:
# Interseção 6 - Interseção 5 + Amazônia Legal
for ano in anos:
    # Caminho dos vetores de Interseção 5
    int = glob.glob(temp_path + fr'**/*{ano}*UCPI_CAR.shp')
    # Caminho dos vetores de Amazônia Legal
    base = glob.glob(amz_path + '**/*.shp')
    caminho_int = []
    caminho_base = []
    for arquivo in int:
        caminho = os.path.join(temp_path,arquivo)
        caminho_int.append(caminho)
    for arquivo in base:
        caminho = os.path.join(ti_path,arquivo)
        caminho_base.append(caminho)
    vetores = caminho_int + caminho_base
    for municipio in municipios:
        for vetor in vetores:
            if municipio in vetor and 'Usos' in vetor:
                nome_arq = os.path.basename(vetor).replace('CAR.shp', 'AMZ_CAR.shp')
                uso = gpd.read_file(vetor)
            if municipio in vetor and 'Amz' in vetor:
                area = gpd.read_file(vetor)
        base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))
# Dissolve
base = glob.glob(temp_path + '**/*AMZ_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    nome_arq = os.path.basename(arquivo)
    diss = arq.dissolve(by=['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_DECLIV', 'APTD_EDAFO', 'CD_EDAFO', 'LEG_SOLO', 'CD_TERRIND', 'CD_UCPI', 'CD_AMZLG'], as_index=True)
    diss.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [ ]:
# Interseção 7 - Interseção 6 + Municípios
for ano in anos:
    # Caminho dos vetores de Interseção 5
    int = glob.glob(temp_path + fr'**/*{ano}*AMZ_CAR.shp')
    municipios = gpd.read_file(mun_path)
    for arquivo in int:
        nome_arq = os.path.basename(arquivo)
        uso = gpd.read_file(arquivo)
        base_car = gpd.overlay(uso, municipios, how='intersection', keep_geom_type=True)
        base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))
# Dissolve
base = glob.glob(temp_path + '**/*AMZ_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    nome_arq = os.path.basename(arquivo)
    diss = arq.dissolve(by=['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_DECLIV', 'APTD_EDAFO', 'CD_EDAFO', 'LEG_SOLO', 'CD_TERRIND', 'CD_UCPI', 'CD_AMZLG', 'CD_MUN'], as_index=True)
    diss.to_file(driver = 'ESRI Shapefile', filename = (rf'{temp_path}\{nome_arq}'))

In [9]:
# Criar pasta para armazenar arquivos finais de Potencial Agropecuário
pot_path = os.path.join(dirpath, 'Potencial_Agropecuario')
os.makedirs(temp_path, exist_ok=True)
# Adequando o arquivo para o Banco de Dados
base = glob.glob(temp_path + '**/*AMZ_CAR.shp')
for arquivo in base:
    arq = gpd.read_file(arquivo)
    arq = arq.drop(['NUM_AREA', 'TIPO_IMOVE', 'SITUACAO', 'CONDICAO_I', 'terrai_cod', 'AMZLG_HA_C', 'NM_MUN', 'SIGLA_UF', 'AREA_KM2'], axis = 1)
    converter_colunas = ['CD_USO', 'CD_DECLIV', 'CD_UCPI']
    arq[converter_colunas] = arq[converter_colunas].astype('object')
    ordem = ['CD_USO', 'COD_IMOVEL', 'COD_ESTADO', 'NOM_MUNICI', 'NUM_MODULO', 'CD_MUN', 'CD_DECLIV', 'APTD_EDAFO', 'CD_EDAFO', 'LEG_SOLO', 'CD_AMZLG', 'CD_UCPI', 'CD_TERRIND', 'geometry']
    arq = arq.reindex(columns=ordem)
    arq.columns = arq.columns.str.lower()
    nome_arq = os.path.basename(arquivo).replace('Usos_ABC', 'Potencial_Agropecuario').replace('_Decl_Aptd_TI_UCPI_AMZ_CAR.shp', '.shp')
    # Adequando o encoding para utf-8
    # Definir coluna a ser ajustada
    coluna = 'nom_munici'
    # Extraindo valores únicos da coluna
    valoresUnicos = arq[coluna].unique()
    for valor in valoresUnicos:
        valor_encode = valor.encode('iso-8859-1')
        valor_decode = str(valor_encode.decode('utf-8'))
        arq.loc[arq[coluna]==f'{valor}',coluna] = f'{valor_decode}'
        arq.to_file(driver = 'ESRI Shapefile', filename = (rf'{pot_path}\{nome_arq}'))